# CUDA exercises

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import itertools
%matplotlib inline

from IPython.display import display

In [ ]:
def csv_parse(path, header=False, sep=";"):
    if header:
        df = pd.read_csv(path, sep=sep, names=header)
    else:
        df = pd.read_csv(path, sep=sep)
    #display(df.head())
    return df

In [ ]:
def calcGB_Stride(df):
    return 3*8* df["N"]/df["k"] /1e9

def calcGB_Offset(df):
    return 3*8* (df["N"]-df["k"])/1e9

def transform_to_ms(df, cols):
    return df[cols]*1e3


# Ex7.1

In [ ]:
def print_styling():

    plt.rc('figure', figsize=(14,8)) 
    plt.style.use("seaborn")
    sns.set_palette("Paired")
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('lines', linewidth=2)

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
print_styling()

In [ ]:
df_CPU_slow = csv_parse("data/ph_data_CPU.csv", header=False)
df_CPU_fast = csv_parse("data/ph_data2_CPU.csv", header=False)
df_GPU_slow = csv_parse("data/ph_data_GPU.csv", header=False)
df_GPU_fast = csv_parse("data/ph_data2_GPU.csv", header=False)

for df_slow, df_fast in zip( [df_CPU_slow, df_GPU_slow],[df_CPU_fast, df_GPU_fast]):
    df_fast["speedup"] = df_slow["ocl_time"] / df_fast["ocl_time"]

data = [df_CPU_slow, df_CPU_fast, df_GPU_slow, df_GPU_fast]
data_fast = [df_CPU_fast, df_GPU_fast]
for df in data:
    display(df)

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(14,10))

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"


print_styling();
for df, ax in zip(data, axs.ravel()):
    target = df["target"].unique()[0]
    labels1 = [f"OpenCL/{target}/laplace", f"Host/laplace"]
    labels2 = [f"OpenCL/{target}/other", f"Host/other"]
    df[df["gen_type"]==1].plot(x="N", y=["ocl_time", "cpu_time"], ax=ax, logx=logx, logy=logy, label=labels1)
    df[df["gen_type"]==2].plot(x="N", y=["ocl_time", "cpu_time"], ax=ax, logx=logx, logy=logy, label=labels2)

#ax.set_ylabel("runtime [s]")
#ax.set_title("Total")
fig.suptitle("Sparse Matrix-Vector Product")
for ax in axs.ravel():
    ax.grid(True)
titles= ["Slow kernel", "Fast kernel"]
for ax, title in zip(axs.ravel()[0:2], titles):
    ax.set_title(title)
# Set ylabels
axs[0][0].set_ylabel("runtime [s]")
axs[1][0].set_ylabel("runtime [s]")
# Set xlabels
axs[0][0].set_xlabel("")
axs[0][1].set_xlabel("")

plt.tight_layout()
fig.savefig("plots/ex7.2_runtimes.pdf", bbox_inches="tight", pad_inches=0.2)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(14,10))

logx, logy = True, False
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"


print_styling();
for df, ax in zip(data_fast, axs.ravel()):
    target = df["target"].unique()[0]
    labels1 = f"OpenCL/{target}/laplace"
    labels2 = f"OpenCL/{target}/other"
    df[df["gen_type"]==1].plot(x="N", y="speedup", ax=ax, logx=logx, logy=logy, label=labels1)
    df[df["gen_type"]==2].plot(x="N", y="speedup", ax=ax, logx=logx, logy=logy, label=labels2)

#ax.set_ylabel("runtime [s]")
#ax.set_title("Total")
fig.suptitle("Sparse Matrix-Vector Product")
for ax in axs.ravel():
    ax.grid(True)
titles= ["CPU", "GPU"]
for ax, title in zip(axs.ravel()[0:2], titles):
    ax.set_title(title)
# Set ylabels
axs[0].set_ylabel("runtime [s]")

plt.tight_layout()
fig.savefig("plots/ex7.2_speedup.pdf", bbox_inches="tight", pad_inches=0.2)